In [ ]:
import osmnx as ox # OSMnx is a Python package to get access to geospatial features from OpenStreetMap. (Boeing, G. 2024)

## 6. POI Counts: 
# Select the number of transportation and shops in the area, 
# as they might act as crime attractors and influence crime opportunities.

# read lsoa 2011 boundaries
lsoa_gdf = gpd.read_file("https://raw.githubusercontent.com/meimao76/006assessment/refs/heads/master/data/statistical-gis-boundaries-london/statistical-gis-boundaries-london/ESRI/LSOA_2011_London_gen_MHW.shp")

# get poi data from open street map
# code from osmnx website
place = "London, UK"
tags1 = {"railway": "station", "highway": "bus_stop"}  # subway stations and bus stations
tags2 = {"shop": True } # shops
gdf1 = ox.features_from_place(place, tags1)
gdf2 = ox.features_from_place(place, tags2)

# set the same crs
tran_poi_gdf = gdf1.to_crs(lsoa_gdf.crs)
shop_poi_gdf = gdf2.to_crs(lsoa_gdf.crs)

# change all types of geospatial features into points
shop_poi_gdf["geometry"] = shop_poi_gdf.geometry.centroid

# join the transportation points with the lsoas
joined_tran = gpd.sjoin(tran_poi_gdf, lsoa_gdf, how='inner', predicate='within')
# counts the points within each lsoa
tran_poi_count = joined_tran.groupby('LSOA11CD').size().reset_index(name='tran_poi_count')
tran_poi_count = tran_poi_count.rename(columns={"LSOA11CD": "LSOA Code"})

# same with the shop points
joined_shop = gpd.sjoin(shop_poi_gdf, lsoa_gdf, how='inner', predicate='within')
shop_poi_count = joined_shop.groupby('LSOA11CD').size().reset_index(name='shop_poi_count')
shop_poi_count = shop_poi_count.rename(columns={"LSOA11CD": "LSOA Code"})

print(tran_poi_count.head(2))
print(shop_poi_count.head(2))